In [22]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import pandana as pdn
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
import warnings
import os
import json

warnings.filterwarnings("ignore")

In [23]:
pois = gpd.read_file('/app/data/input/amenities_costanera')

In [32]:
def load_pandana_network_graph():
    graph = ox.graph_from_place(
        "Concepción, Chile",
        network_type="walk",
        simplify=False,
    )

    nodes_df, edges_df = ox.graph_to_gdfs(graph)
    nodes_df = nodes_df.rename(columns={'x': 'lon', 'y': 'lat'})
    nodes_df['id'] = nodes_df.index
    edges_df['from'] = edges_df.reset_index()['u'].values
    edges_df['to'] = edges_df.reset_index()['v'].values

    net = pdn.Network(
        nodes_df['lon'],
        nodes_df['lat'],
        edges_df['from'],
        edges_df['to'],
        edges_df[['length']]
    )
    return net

net = load_pandana_network_graph()

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 37494
Setting CH edge vector of size 82736
Range graph removed 82736 edges of 165472
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [41]:
grid = gpd.read_file('/app/data/input/grilla')
grid = grid.to_crs(pois.crs)

In [42]:
# Agrega el centroide a la grilla y los puntos (x,y)
grid['centroid'] = grid['geometry'].centroid
grid['x'] = grid['centroid'].x
grid['y'] = grid['centroid'].y

# Agrega los puntos (x,y) de los pois
pois['x'] = pois['geometry'].x
pois['y'] = pois['geometry'].y

In [44]:
grid['nearest_node'] = net.get_node_ids(grid['x'], grid['y'])
pois['nearest_node'] = net.get_node_ids(pois['x'], pois['y'])

In [47]:
n_rank = 3
output_cols = ['grid_id', 'rank', 'origin', 'destination', 'path_lengths', 'Category', 'Subcategor', 'geometry']

categories = pois['Category'].unique()
df_output = []

for category in categories:
    mask = pois['Category']==category
    destination_nodes = pois.loc[mask, 'nearest_node']
    for index, row in grid.iterrows():
        origin_node = row['nearest_node']
        origin_nodes = [origin_node]*len(destination_nodes)
        shortest_path_lengths = net.shortest_path_lengths(origin_nodes, destination_nodes)
        df = pd.DataFrame.from_dict({
            'origin': origin_nodes,
            'destination': destination_nodes,
            'path_lengths': shortest_path_lengths,
        })
        df.sort_values(by='path_lengths', inplace=True)
        df = df.head(n_rank)
        df['grid_id'] = grid.loc[index, 'grid_id']
        df.reset_index(inplace=True, drop=True)
        df['rank'] = df.index + 1
        df = pd.merge(df, pois[mask].rename(columns={'nearest_node': 'destination'}), on='destination')

        df_output.append(df[output_cols])

df_output = pd.concat(df_output)

In [49]:
df_output

,grid_id,rank,origin,destination,path_lengths,Category,Subcategor,geometry
0,1,1,4187902988,2374420338,1931.169,Comida para servir,cafe,POINT (-73.06442 -36.82969)
1,1,1,4187902988,2374420338,1931.169,Comida para servir,NaN,POINT (-73.06443 -36.82969)
2,1,2,4187902988,2374420338,1931.169,Comida para servir,cafe,POINT (-73.06442 -36.82969)
3,1,2,4187902988,2374420338,1931.169,Comida para servir,NaN,POINT (-73.06443 -36.82969)
4,1,3,4187902988,11198912835,1937.389,Comida para servir,restaurant,POINT (-73.06387 -36.83131)
...,...,...,...,...,...,...,...,...
0,2077,1,5996263458,5441691718,2560.244,Comercio,NaN,POINT (-73.06244 -36.82945)
1,2077,2,5996263458,2374420273,2733.222,Comercio,NaN,POINT (-73.06335 -36.82963)
2,2077,2,5996263458,2374420273,2733.222,Comercio,NaN,POINT (-73.06324 -36.82959)
3,2077,3,5996263458,2374420273,2733.222,Comercio,NaN,POINT (-73.06335 -36.82963)


In [50]:
df_output.to_pickle('/app/data/output/costanera_amenities_accesibility.pkl')

In [51]:
gdf_output = gpd.GeoDataFrame(data=df_output.drop(columns=['geometry']), geometry=df_output['geometry'])

In [52]:
gdf_output.to_file('/app/data/output/costanera_amenities_accesibility')